In [1]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

In [2]:
    classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
            "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
    # config file 들고오기
    cfg = Config.fromfile('/opt/ml/code/mmdetection_trash/configs/_base_/models/swim_base_htc.py')

    PREFIX = '../../input/data/'
    cfg.runner.max_epochs = 60

  

    # dataset 바꾸기
    cfg.data.train.classes = classes
    cfg.data.train.img_prefix = PREFIX
    cfg.data.train.seg_prefix = PREFIX
    cfg.data.train.ann_file = PREFIX + 'train_data3.json'
    #cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

    cfg.data.val.classes = classes
    cfg.data.val.img_prefix = PREFIX
    cfg.data.val.ann_file = PREFIX + 'valid_data3.json'
    #cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

    cfg.data.test.classes = classes
    cfg.data.test.img_prefix = PREFIX
    cfg.data.test.ann_file = PREFIX + 'test.json'
    #cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

    cfg.data.samples_per_gpu = 2

    # cfg.optimizer = dict(type='AdamW', lr=0.0001, weight_decay=0.0001)
    # cfg.lr_config = dict(
    #     policy='CosineRestart',
    #     warmup='linear',
    #     warmup_iters=200,
    #     warmup_ratio=1.0 / 10,
    #     periods=[20,12,12,12,12],
    #     restart_weights = [1,0.7,0.6,0.5,0.4],
    #     min_lr_ratio=2e-6)

    cfg.seed= 24
    cfg.gpu_ids = [0]
    cfg.work_dir = './work_dirs/swim_b_htc_01_losschange'
    cfg.log_config.interval=100

    cfg.checkpoint_config = dict(max_keep_ckpts=2, interval=1)
    #cfg.model.roi_head.bbox_head.num_classes = 11

    #이어하기
    #cfg.resume_from = './work_dirs/swin_t_mask_rcnn/epoch_15.pth'
    cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

    #group_name = '아무거나'; ##그룹이름(run 여러개를 그룹으로 묶어줄 수 있습니다)
#     project_name = 'stage_3_object_det'; ##프로젝트 이름
#     run_name = 'swim_b_htc_01_losschange'; ##run 이름stage_3_object_det
#     config_list = {
#         'epoch' : cfg.runner.max_epochs,
#         'batch_size' :  cfg.data.samples_per_gpu,
#         'optimizer' : cfg.optimizer,
#         'optimizer_config' : cfg.optimizer_config,
#         'lr_config' : cfg.lr_config
#     }
    ## 원하는 wandb.init() argument값 추가해주세요
    #cfg.log_config.hooks[1].init_kwargs['group']=group_name   # group name(option)
#     cfg.log_config.hooks[1].init_kwargs['project'] = project_name
#     cfg.log_config.hooks[1].init_kwargs['name'] = run_name    # run name
#     cfg.log_config.hooks[1].init_kwargs['config'] = config_list    # config

In [3]:
model = build_detector(cfg.model)

2021-05-19 15:56:23,894 - mmdet - INFO - load model from: /opt/ml/code/mmdetection_trash/work_dirs/swin/swin_base_patch4_window7_224_22k.pth
2021-05-19 15:56:24,925 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: norm.weight, norm.bias, head.weight, head.bias, layers.0.blocks.1.attn_mask, layers.1.blocks.1.attn_mask, layers.2.blocks.1.attn_mask, layers.2.blocks.3.attn_mask, layers.2.blocks.5.attn_mask, layers.2.blocks.7.attn_mask, layers.2.blocks.9.attn_mask, layers.2.blocks.11.attn_mask, layers.2.blocks.13.attn_mask, layers.2.blocks.15.attn_mask, layers.2.blocks.17.attn_mask

missing keys in source state_dict: norm0.weight, norm0.bias, norm1.weight, norm1.bias, norm2.weight, norm2.bias, norm3.weight, norm3.bias



In [4]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=3.41s)
creating index...
index created!


/opt/conda/lib/python3.7/site-packages/albumentations/augmentations/transforms.py:2611: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  "blur_limit and sigma_limit minimum value can not be both equal to 0. "


In [5]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...


2021-05-19 15:56:32,361 - mmdet - INFO - Start running, host: root@fca80b852140, work_dir: /opt/ml/code/mmdetection_trash/work_dirs/swim_b_htc_01_losschange
2021-05-19 15:56:32,361 - mmdet - INFO - workflow: [('train', 1)], max: 60 epochs


Done (t=0.79s)
creating index...
index created!


2021-05-19 15:58:47,289 - mmdet - INFO - Epoch [1][100/1313]	lr: 9.909e-06, eta: 1 day, 5:29:10, time: 1.349, data_time: 0.055, memory: 15581, loss_rpn_cls: 0.6573, loss_rpn_bbox: 0.0479, loss_semantic_seg: 1.4450, s0.loss_cls: 1.0777, s0.acc: 82.4580, s0.loss_bbox: 0.0225, s0.loss_mask: 2.4290, s1.loss_cls: 0.3525, s1.acc: 91.5605, s1.loss_bbox: 0.0056, s1.loss_mask: 1.9398, s2.loss_cls: 0.2368, s2.acc: 84.6855, s2.loss_bbox: 0.0007, s2.loss_mask: 1.0519, loss: 9.2667, grad_norm: 128.0842
2021-05-19 16:01:18,248 - mmdet - INFO - Epoch [1][200/1313]	lr: 1.991e-05, eta: 1 day, 7:12:00, time: 1.510, data_time: 0.020, memory: 15908, loss_rpn_cls: 0.3594, loss_rpn_bbox: 0.0432, loss_semantic_seg: 0.3531, s0.loss_cls: 0.7041, s0.acc: 91.9736, s0.loss_bbox: 0.1903, s0.loss_mask: 0.7662, s1.loss_cls: 0.2721, s1.acc: 96.2617, s1.loss_bbox: 0.0598, s1.loss_mask: 0.4259, s2.loss_cls: 0.1163, s2.acc: 98.0098, s2.loss_bbox: 0.0096, s2.loss_mask: 0.2035, loss: 3.5036, grad_norm: 22.9085
2021-05-19 

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 647/647, 2.8 task/s, elapsed: 228s, ETA:     0s

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices